In [1]:
import numpy as np 
import tensorflow as tf 
import pdb 

In [2]:
def basic_MLP(X,sizes,is_training,bits=8,dropout=False): 
    layers = {}
    layers[0] = tf.layers.dense(X, sizes[0],
                                activation = tf.nn.relu,
                                use_bias=False,
                                )
    if dropout:
        dropout_key = str(0)+'d' 
        layers[dropout_key] = tf.layers.dropout(layers[0],
                                                rate=0.25,
                                                training=is_training)
    for j,  size in enumerate(sizes[1:]): 
        i = j+1
        if dropout: 
            dropout_key_prior = str(i-1)+'d' 
            layers[i] = tf.layers.dense(layers[dropout_key_prior], size,
                                    activation = tf.nn.relu,
                                    use_bias=True,
                                    )
            dropout_key = str(i)+'d' 
            layers[dropout_key] = tf.layers.dropout(layers[dropout_key],
                                                rate=0.25,
                                                training=is_training)
        else:
            layers[i] = tf.layers.dense(layers[i-1], size,
                                    activation = tf.nn.relu,
                                    use_bias=True,
                                    )

    binary = tf.layers.dense(layers[len(sizes)-1], bits,
                                   activation = tf.tanh,
                                   use_bias=True,
                                    ) 
    
    binary += 1.0
    binary *= 0.5
    
    output = tf.layers.dense(binary, 1,
                            activation = None,
                            use_bias=False,
                                    ) 
    return [output, binary]


In [9]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, 1])
Y = tf.placeholder(tf.float32, [None, 1])
is_training = tf.placeholder(tf.bool)

sizes = [512, 512, 512]
output, binary = basic_MLP(X,sizes,is_training,bits=16,dropout=False)

binary_loss = tf.reduce_mean((binary**3 - binary)**2)
trans_loss = tf.reduce_mean((output - Y)**2)

loss = trans_loss + 0.1*binary_loss

optimizer = tf.train.AdamOptimizer(1e-2) 
train_step = optimizer.minimize(loss)

In [14]:
X_train = np.arange(-1.5, 1.5, 0.01)
X_train = X_train.reshape([-1,1])
feed_dict = {X:X_train, Y:X_train, is_training:True}

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
for i in range(1000):
    _, b_loss, t_loss = sess.run([train_step, binary_loss, trans_loss], feed_dict)
    if (i+1)%100 == 0:
        print('Binary Loss:', b_loss)
        print('Translation Loss:', t_loss)

Binary Loss: 0.0271673
Translation Loss: 0.00150883
Binary Loss: 0.0101796
Translation Loss: 0.000181143
Binary Loss: 0.0065316
Translation Loss: 0.000212362
Binary Loss: 0.00569265
Translation Loss: 6.87164e-05
Binary Loss: 0.00534498
Translation Loss: 0.000160041
Binary Loss: 0.00525592
Translation Loss: 0.000426314
Binary Loss: 0.00518281
Translation Loss: 8.23801e-05
Binary Loss: 0.00504495
Translation Loss: 3.47351e-05
Binary Loss: 0.00497606
Translation Loss: 3.32131e-05
Binary Loss: 0.00491436
Translation Loss: 5.97514e-05


In [19]:
error = []
for i in range(X_train.shape[0]):
    Xd = X_train[i,0].reshape([1,1])
    feed_dict = {X:Xd, Y:Xd, is_training:False}
    b_loss, t_loss = sess.run([binary_loss, trans_loss], feed_dict)
    error.append(t_loss)
max_error = np.amax(error)
print('Maximum error:',max_error)

Maximum error: 0.000829715
